In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from google.colab import drive

In [2]:
# Mount drive and import files
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# define filepath
file_path = "/content/drive/MyDrive/Sentiment_Data/cleaned_sentinment_1.csv"

In [37]:
sentiment_df = pd.read_csv(file_path)
sentiment_df.head()

,Unnamed: 0,ticker,date,overall_sent_comp,overall_sent_pos,overall_sent_neg,mean_t_comp_score,mean_t_pos_score,mean_t_neg_score,mean_tgt_comp_score,...,CloseOver60,VolumeOver60,CloseOver90,VolumeOver90,Co30_d,Co60_d,Co90_d,Vo30_d,Vo60_d,Vo90_d
0,0,DIS,2020-08-11,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
1,1,DIS,2020-08-11,0.8658,0.245,0.036,0.176100,0.164667,0.0740,0.7783,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
2,2,DIS,2020-08-11,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
3,3,DIS,2020-08-11,0.5078,0.102,0.075,0.087067,0.117833,0.0485,-0.5818,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0
4,4,DIS,2020-08-11,-0.5994,0.000,0.438,-0.599400,0.000000,0.4380,-0.5994,...,127.534833,8.716522e+06,135.033889,10756920.0,1,0,1,0,0,0


In [38]:
len(sentiment_df.index)

698140

In [39]:
sentiment_df = sentiment_df.drop(columns='Unnamed: 0')

In [40]:
sentiment_df.columns

Index(['ticker', 'date', 'overall_sent_comp', 'overall_sent_pos',
       'overall_sent_neg', 'mean_t_comp_score', 'mean_t_pos_score',
       'mean_t_neg_score', 'mean_tgt_comp_score', 'mean_tgt_pos_score',
       'mean_tgt_neg_score', 'verb_tense', 'mean_custom_score', 'Close',
       'Volume', 'CloseOver30', 'VolumeOver30', 'CloseOver60', 'VolumeOver60',
       'CloseOver90', 'VolumeOver90', 'Co30_d', 'Co60_d', 'Co90_d', 'Vo30_d',
       'Vo60_d', 'Vo90_d'],
      dtype='object')

In [41]:
# sanitize and convert to numeric
#sent_comp = pd.to_numeric(sentiment_df['overall_sent_comp'], errors='coerce')
#sentiment_df['overall_sent_comp'] = sent_comp
#sentiment_df = sentiment_df.dropna(how='any')
#sentiment_df = sentiment_df.astype('float64')

In [42]:
# Set up the different target arrays
y_1 = sentiment_df['Co30_d']
y_2 = sentiment_df['Co60_d']
y_3 = sentiment_df['Co90_d']

#verb_tense = sentiment_df['verb_tense']

X = sentiment_df.drop(columns=['Close', 'ticker', 'date', 'verb_tense',
       'Volume', 'CloseOver30', 'VolumeOver30', 'CloseOver60', 'VolumeOver60',
       'CloseOver90', 'VolumeOver90', 'Co30_d', 'Co60_d', 'Co90_d', 'Vo30_d',
       'Vo60_d', 'Vo90_d'])


X = X.astype('float64')
# add verb_tense field back in
#X['verb_tense'] = verb_tense
#X = pd.get_dummies(sentiment_df)

#sent_comp = pd.to_numeric(X['overall_sent_comp'], errors='coerce')

In [43]:
X.dtypes

overall_sent_comp      float64
overall_sent_pos       float64
overall_sent_neg       float64
mean_t_comp_score      float64
mean_t_pos_score       float64
mean_t_neg_score       float64
mean_tgt_comp_score    float64
mean_tgt_pos_score     float64
mean_tgt_neg_score     float64
mean_custom_score      float64
dtype: object

In [44]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_1)

In [45]:
len(list(X_train.head(n=1)))

10

In [46]:
# All data should already be scaled due to sentiment generated value thresholds
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(list(X_train.head(n=1)))
hidden_nodes_layer1 = 5
#hidden_nodes_layer2 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 5)                 55        
                                                                 
 dense_16 (Dense)            (None, 1)                 6         
                                                                 
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [48]:
# Train the model
fit_model = nn.fit(X_train,
                   y_train,
                   epochs=10)

Epoch 1/10
16363/16363 [==============================] - 29s 2ms/step - loss: 0.6685 - accuracy: 0.5995
Epoch 2/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6664 - accuracy: 0.6046
Epoch 3/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6662 - accuracy: 0.6045
Epoch 4/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6661 - accuracy: 0.6044
Epoch 5/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6660 - accuracy: 0.6044
Epoch 6/10
16363/16363 [==============================] - 29s 2ms/step - loss: 0.6660 - accuracy: 0.6044
Epoch 7/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6660 - accuracy: 0.6045
Epoch 8/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6660 - accuracy: 0.6045
Epoch 9/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6660 - accuracy: 0.6045
Epoch 10/10
16363/16363 [==============================

In [49]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5455/5455 - 9s - loss: 0.6671 - accuracy: 0.6027 - 9s/epoch - 2ms/step
Loss: 0.6670926213264465, Accuracy: 0.6026527881622314


In [50]:
# Testing out N60 
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_2)

In [51]:
# All data should already be scaled due to sentiment generated value thresholds
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(list(X_train.head(n=1)))
hidden_nodes_layer1 = 5
#hidden_nodes_layer2 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 5)                 55        
                                                                 
 dense_18 (Dense)            (None, 1)                 6         
                                                                 
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [53]:
# Train the model
fit_model = nn.fit(X_train,
                   y_train,
                   epochs=10)

Epoch 1/10
16363/16363 [==============================] - 29s 2ms/step - loss: 0.6763 - accuracy: 0.5850
Epoch 2/10
16363/16363 [==============================] - 30s 2ms/step - loss: 0.6751 - accuracy: 0.5853
Epoch 3/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6749 - accuracy: 0.5862
Epoch 4/10
16363/16363 [==============================] - 29s 2ms/step - loss: 0.6748 - accuracy: 0.5861
Epoch 5/10
16363/16363 [==============================] - 32s 2ms/step - loss: 0.6747 - accuracy: 0.5859
Epoch 6/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6746 - accuracy: 0.5857
Epoch 7/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6746 - accuracy: 0.5858
Epoch 8/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6746 - accuracy: 0.5867
Epoch 9/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6746 - accuracy: 0.5862
Epoch 10/10
16363/16363 [==============================

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5455/5455 - 10s - loss: 0.6743 - accuracy: 0.5855 - 10s/epoch - 2ms/step
Loss: 0.6743292212486267, Accuracy: 0.5855215191841125


In [59]:
# Testing out N90days
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_3)

In [60]:
# All data should already be scaled due to sentiment generated value thresholds
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(list(X_train.head(n=1)))
hidden_nodes_layer1 = 5
#hidden_nodes_layer2 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 5)                 55        
                                                                 
 dense_22 (Dense)            (None, 1)                 6         
                                                                 
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [62]:
# Train the model
fit_model = nn.fit(X_train,
                   y_train,
                   epochs=10)

Epoch 1/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6513 - accuracy: 0.6426
Epoch 2/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6502 - accuracy: 0.6437
Epoch 3/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6501 - accuracy: 0.6437
Epoch 4/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6501 - accuracy: 0.6437
Epoch 5/10
16363/16363 [==============================] - 29s 2ms/step - loss: 0.6501 - accuracy: 0.6437
Epoch 6/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6500 - accuracy: 0.6437
Epoch 7/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6500 - accuracy: 0.6437
Epoch 8/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6500 - accuracy: 0.6437
Epoch 9/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6500 - accuracy: 0.6437
Epoch 10/10
16363/16363 [==============================

In [63]:
X.columns

Index(['overall_sent_comp', 'overall_sent_pos', 'overall_sent_neg',
       'mean_t_comp_score', 'mean_t_pos_score', 'mean_t_neg_score',
       'mean_tgt_comp_score', 'mean_tgt_pos_score', 'mean_tgt_neg_score',
       'mean_custom_score'],
      dtype='object')

In [65]:
# testing smaller set of sentiment columns
X = X[['overall_sent_comp', 'mean_t_comp_score', 'mean_tgt_comp_score', 'mean_custom_score']]

In [66]:
# Testing out N90days
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_3)

In [67]:
# All data should already be scaled due to sentiment generated value thresholds
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(list(X_train.head(n=1)))
hidden_nodes_layer1 = 3
#hidden_nodes_layer2 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 3)                 15        
                                                                 
 dense_24 (Dense)            (None, 1)                 4         
                                                                 
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________


In [68]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [69]:
# Train the model
fit_model = nn.fit(X_train,
                   y_train,
                   epochs=10)

Epoch 1/10
16363/16363 [==============================] - 31s 2ms/step - loss: 0.6512 - accuracy: 0.6443
Epoch 2/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6502 - accuracy: 0.6443
Epoch 3/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6502 - accuracy: 0.6443
Epoch 4/10
16363/16363 [==============================] - 29s 2ms/step - loss: 0.6502 - accuracy: 0.6443
Epoch 5/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6502 - accuracy: 0.6443
Epoch 6/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6501 - accuracy: 0.6443
Epoch 7/10
16363/16363 [==============================] - 28s 2ms/step - loss: 0.6501 - accuracy: 0.6443
Epoch 8/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6501 - accuracy: 0.6443
Epoch 9/10
16363/16363 [==============================] - 27s 2ms/step - loss: 0.6501 - accuracy: 0.6443
Epoch 10/10
16363/16363 [==============================

In [70]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5455/5455 - 7s - loss: 0.6510 - accuracy: 0.6425 - 7s/epoch - 1ms/step
Loss: 0.6510362029075623, Accuracy: 0.6424556970596313
